In [ ]:
# for cloud computing
# !pip install Flask-PyMongo
# !pip install praw
# !pip install nba-api

In [3]:
#data wrangling
import pandas as pd
import requests
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

#profanity testing
from profanity_check import predict, predict_prob

#time
import time
from datetime import datetime as dt

#db
import pymongo
from flask_pymongo import PyMongo

#api
import praw
import nba_api
from nba_api.stats.static import teams
from nba_api.stats.endpoints import LeagueGameLog
from nba_api.stats.endpoints import leaguegamefinder

#keys for reddit api
from resources import r_clientid, r_clientsecret, r_username, r_password, r_useragent

#mongoDB keys
from resources import mongo_user, mongo_pass, mongo_clusteruri

In [4]:
r_clientid

'rJmTVJhI8RHW2A'

In [6]:
#mongo db initialization/test
conn = (f'mongodb+srv://{mongo_user}:{mongo_pass}@{mongo_clusteruri}')
client = pymongo.MongoClient(conn)

db = client.nba_db
collection = db.nba_reddit.find()

In [ ]:
#verificaiton
document = (
    {
        'id_submission': 15254,
        'created_utc': '2019-04-10 23:49',
        'title': 'fake submission',
        'url': 'https:fake.url',
        'top_comment': 
            {'id_comment': 'u141v3',
            'author': 'u/razzdrazz',
            'body': 'This is a fake comment',
            'score': 48,
            'created_utc': 'April 10, 2019',
            'permalink': 'https:fake.url',
            'expletive_score': 0
            },
    }
)

#replace_one upsert=True method
db.nba_reddit.replace_one(document, document, True)

In [7]:
collection = db.nba_reddit.find()

#verificaiton
for doc in collection:
    pp.pprint(doc)

{   '_id': ObjectId('5d7ed733bb07e9f5f70c08de'),
    'created_utc': '2019-04-10 23:49',
    'id_submission': 15254,
    'title': 'fake submission',
    'top_comment': {   'author': 'u/razzdrazz',
                       'body': 'This is a fake comment',
                       'created_utc': 'April 10, 2019',
                       'expletive_score': 0,
                       'id_comment': 'u141v3',
                       'permalink': 'https:fake.url',
                       'score': 48},
    'url': 'https:fake.url'}
{   '_id': ObjectId('5d7eee96bb07e9f5f7100400'),
    'away_plusminus': -18,
    'away_pts': 87,
    'away_stats': {   'away_ast': 18,
                      'away_blk': 5,
                      'away_dreb': 41,
                      'away_fg3_pct': 0.192,
                      'away_fg3a': 26,
                      'away_fg3m': 5,
                      'away_fg_pct': 0.391,
                      'away_fga': 87,
                      'away_fgm': 34,
                      'away

    'away_stats': {   'away_ast': 24,
                      'away_blk': 2,
                      'away_dreb': 30,
                      'away_fg3_pct': 0.4,
                      'away_fg3a': 25,
                      'away_fg3m': 10,
                      'away_fg_pct': 0.521,
                      'away_fga': 96,
                      'away_fgm': 50,
                      'away_ft_pct': 0.731,
                      'away_fta': 26,
                      'away_ftm': 19,
                      'away_mins': 240,
                      'away_oreb': 9,
                      'away_pf': 26,
                      'away_reb': 39,
                      'away_stl': 6,
                      'away_tov': 13},
    'away_wl': 'L',
    'awayteam': 'Sacramento Kings',
    'game_date': '2018-10-19',
    'game_id': '0021800022',
    'game_thread': {   'created_utc': '2018-10-19 23:14',
                       'id_submission': '9pp0xy',
                       'num_comments': 443,
                       'titl

                      'home_fg3m': 11,
                      'home_fg_pct': 0.398,
                      'home_fga': 108,
                      'home_fgm': 43,
                      'home_ft_pct': 0.871,
                      'home_fta': 31,
                      'home_ftm': 27,
                      'home_mins': 265,
                      'home_oreb': 17,
                      'home_pf': 22,
                      'home_reb': 70,
                      'home_stl': 6,
                      'home_tov': 20},
    'home_wl': 'L',
    'hometeam': 'Portland Trail Blazers',
    'matchup': 'POR vs. WAS',
    'season': '22018'}
{   '_id': ObjectId('5d7eef79bb07e9f5f7102c25'),
    'away_plusminus': -21,
    'away_pts': 106,
    'away_stats': {   'away_ast': 21,
                      'away_blk': 5,
                      'away_dreb': 27,
                      'away_fg3_pct': 0.321,
                      'away_fg3a': 28,
                      'away_fg3m': 9,
                      'away_fg_pct': 0.423

                       'num_comments': 498,
                       'title': 'GAME THREAD: Milwaukee Bucks (4-0) @ '
                                'Minnesota Timberwolves (2-3) - (October 26, '
                                '2018)',
                       'top_comment': {   'author': 'bootum',
                                          'body': 'KAT get your shit together '
                                                  'dude I’m not making another '
                                                  'video for you already ',
                                          'created_utc': '2018-10-27 00:50',
                                          'expletive_score': 1,
                                          'id_comment': 'e8iu3bx',
                                          'permalink': '/r/nba/comments/9rpoyg/game_thread_milwaukee_bucks_40_minnesota/e8iu3bx/',
                                          'score': 41},
                       'url': 'https://www.reddit.com/r/nba/comments/9

                      'away_reb': 50,
                      'away_stl': 6,
                      'away_tov': 17},
    'away_wl': 'L',
    'awayteam': 'Detroit Pistons',
    'game_date': '2018-10-30',
    'game_id': '0021800099',
    'game_thread': {   'created_utc': '2018-10-30 23:25',
                       'id_submission': '9sta2c',
                       'num_comments': 478,
                       'title': 'GAME THREAD: Detroit Pistons (4-1) @ Boston '
                                'Celtics (4-2) - (October 30, 2018)',
                       'top_comment': {   'author': 'GeneralShepard',
                                          'body': 'Do the refs not realize the '
                                                  'common denominator in all '
                                                  'of these plays is Blake '
                                                  'Griffin being reckless',
                                          'created_utc': '2018-10-31 00:39',
          

...

    'away_pts': 99,
    'away_stats': {   'away_ast': 26,
                      'away_blk': 6,
                      'away_dreb': 37,
                      'away_fg3_pct': 0.419,
                      'away_fg3a': 31,
                      'away_fg3m': 13,
                      'away_fg_pct': 0.435,
                      'away_fga': 92,
                      'away_fgm': 40,
                      'away_ft_pct': 0.545,
                      'away_fta': 11,
                      'away_ftm': 6,
                      'away_mins': 240,
                      'away_oreb': 8,
                      'away_pf': 17,
                      'away_reb': 45,
                      'away_stl': 4,
                      'away_tov': 12},
    'away_wl': 'L',
    'awayteam': 'Indiana Pacers',
    'game_date': '2019-03-27',
    'game_id': '0021801119',
    'game_thread': {   'created_utc': '2019-03-27 23:00',
                       'id_submission': 'b6b5jq',
                       'num_comments': 375,
         

                      'away_fg3_pct': 0.395,
                      'away_fg3a': 38,
                      'away_fg3m': 15,
                      'away_fg_pct': 0.475,
                      'away_fga': 101,
                      'away_fgm': 48,
                      'away_ft_pct': 0.864,
                      'away_fta': 22,
                      'away_ftm': 19,
                      'away_mins': 240,
                      'away_oreb': 11,
                      'away_pf': 20,
                      'away_reb': 46,
                      'away_stl': 16,
                      'away_tov': 11},
    'away_wl': 'W',
    'awayteam': 'Lakers',
    'game_date': '2019-03-31',
    'game_id': '0021801147',
    'game_thread': {   'created_utc': '2019-03-31 23:02',
                       'id_submission': 'b7u103',
                       'num_comments': 29,
                       'title': 'GAME THREAD: Los Angeles Lakers (34-42) @ New '
                                'Orleans Pelicans (32-45) - (March 

    'home_wl': 'L',
    'hometeam': 'Detroit Pistons',
    'matchup': 'DET vs. IND',
    'season': '22018'}
{   '_id': ObjectId('5d7efdc2bb07e9f5f713351a'),
    'away_plusminus': 32,
    'away_pts': 135,
    'away_stats': {   'away_ast': 22,
                      'away_blk': 7,
                      'away_dreb': 36,
                      'away_fg3_pct': 0.474,
                      'away_fg3a': 38,
                      'away_fg3m': 18,
                      'away_fg_pct': 0.534,
                      'away_fga': 88,
                      'away_fgm': 47,
                      'away_ft_pct': 0.793,
                      'away_fta': 29,
                      'away_ftm': 23,
                      'away_mins': 240,
                      'away_oreb': 12,
                      'away_pf': 29,
                      'away_reb': 48,
                      'away_stl': 6,
                      'away_tov': 16},
    'away_wl': 'W',
    'awayteam': 'Houston Rockets',
    'game_date': '2019-04-03',
   

                      'away_pf': 17,
                      'away_reb': 35,
                      'away_stl': 11,
                      'away_tov': 11},
    'away_wl': 'L',
    'awayteam': 'Utah Jazz',
    'game_date': '2019-04-07',
    'game_id': '0021801208',
    'game_thread': {   'created_utc': '2019-04-08 01:23',
                       'id_submission': 'banp2s',
                       'num_comments': 268,
                       'title': 'GAME THREAD: Utah Jazz (49-30) @ Los Angeles '
                                'Lakers (36-44) - (April 07, 2019)',
                       'top_comment': {   'author': 'WarEagle33x',
                                          'body': 'Wtf, I’ve had the game on '
                                                  'in the background with no '
                                                  'sound. I’ve spent the '
                                                  'entire first half cheering '
                                                  'for the

                                          'score': 27},
                       'url': 'https://www.reddit.com/r/nba/comments/bbu8nm/game_thread_sacramento_kings_3942_portland_trail/'},
    'home_plusminus': 5,
    'home_pts': 136,
    'home_stats': {   'home_ast': 19,
                      'home_blk': 2,
                      'home_dreb': 30,
                      'home_fg3_pct': 0.583,
                      'home_fg3a': 24,
                      'home_fg3m': 14,
                      'home_fg_pct': 0.582,
                      'home_fga': 91,
                      'home_fgm': 53,
                      'home_ft_pct': 0.727,
                      'home_fta': 22,
                      'home_ftm': 16,
                      'home_mins': 240,
                      'home_oreb': 14,
                      'home_pf': 17,
                      'home_reb': 44,
                      'home_stl': 5,
                      'home_tov': 13},
    'home_wl': 'W',
    'hometeam': 'Portland Trail Blazers',


# nba api req.
___

In [ ]:
teams_url = 'http://data.nba.net/10s/prod/v2/2019/teams.json'
teams = requests.get(teams_url)
teams

In [ ]:
teams = teams.json()
pp.pprint(teams)

In [ ]:
test = teams['league']['standard']
for team in test:
    if team['isNBAFranchise'] == True:
        pp.pprint(f"{team['fullName']}, {team['teamId']}")

# nba-api wrapper
___

In [ ]:
#teams
nba_teams = teams.get_teams()
nba_teams

In [ ]:
#gamelog
log_games = LeagueGameLog()
df_gamelog = log_games.get_data_frames()[0]
df_gamelog

In [ ]:
df_gamelog.to_csv('gamelog_2019.csv')

In [ ]:
#combine function from api wrapper docs

def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [ ]:
# Combine the game rows (home == TEAM_A, away == TEAM_B)
df_games = combine_team_games(df_gamelog)

#reset index
df_games.reset_index(drop=True, inplace=True)

#save
df_games.to_csv('gamelog_2019_combined.csv')

df_games

In [ ]:
df_games[['TEAM_NAME_A','TEAM_NAME_B']] = df_games[['TEAM_NAME_A','TEAM_NAME_B']].replace('LA Clippers','Clippers')
df_games[['TEAM_NAME_A','TEAM_NAME_B']] = df_games[['TEAM_NAME_A','TEAM_NAME_B']].replace('Los Angeles Lakers','Lakers')

df_games

In [ ]:
#query individual games/individual teams 
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=1610612737)

#keep first df
atl_games = gamefinder.get_data_frames()[0]
atl_games

# balldontlie.io api
___

#### Use this api if official nba or wrapper unsuccessful

In [ ]:
#test w/ kings
games_ball_url = 'https://www.balldontlie.io/api/v1/games?seasons[]=2018&per_page=100&team_ids[]=26'
games_ball = requests.get(games_ball_url).json()
games_ball

# praw (reddit)
___

#### -Praw initialized on the user level--requires client id, secret, username, passord, user agent.
#### -Pull game thread for each corresponding NBA game for the 2018-19 Regular Season, store various fields in dataframe, include rudimentary expletive detection/prediction on top ~25 comments, upload to cloud mongo db.

In [5]:
#initialize and verify user
reddit = praw.Reddit(client_id= r_clientid,
                     client_secret= r_clientsecret,
                     username= r_username,
                     password= r_password,
                     user_agent= r_useragent)

reddit.user.me()

Redditor(name='razzdraz')

In [ ]:
#expletive detection test
print(predict(['1 shit curse word']))
print(predict_prob(['2 crap damn curse words']))

In [ ]:
#choose and verify subreddit
count = 0
sub = 'nba'
subreddit = reddit.subreddit(f'{sub}')

print(f'Top posts in r/{sub} right now')
for submission in subreddit.hot(limit=7):
    
    if submission.stickied == False:
        
        count += 1
        print(f'result {count}:')
        print(submission.title)
        print(f'{submission.score} upvotes')

        #weed out sticky comments
        if submission.comments[0].stickied == True:
            print(f'top comment: {submission.comments[2].body}')
            expl_pct = predict_prob([submission.comments[2].body])
            print(f'expletive prediciton: {expl_pct}')

        elif submission.comments[0].stickied == False: 
            print(f'top comment: {submission.comments[1].body}')
            expl_pct = predict_prob([submission.comments[1].body])
            print(f'expletive prediciton: {expl_pct}')

        print(f'post id: {submission.id}')
        print(submission.url)

        #format unix time --> datetime
        date = dt.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M')
        print(date)
        print(f'stickied: {submission.stickied}')
        print('-------------------------')

In [ ]:
# small df for testing
df_short = df_games[['TEAM_NAME_A','TEAM_NAME_B', 'GAME_DATE','WL_A']].copy()
df_short.reset_index(inplace=True, drop=True)
df_short

In [ ]:
#list category names/indices for itertuples
cats = [i for i in df_games.keys()]
df_cats = pd.DataFrame(cats)
df_cats

In [ ]:
#itertuple method for query

sub = 'nba'
subreddit = reddit.subreddit(f'{sub}')
thread = 'GAME THREAD:'
count_found = 0

# test
# for tup in df_games.head(5).itertuples():

#main loop
for tup in df_games.itertuples():
    count = tup[0]+1
    
    #phrases to search for
    hometeam = tup[4]
    awayteam = tup[32]
    game_date = tup[6]
    
    #other variables from df to store
    game_season = tup[1]
    game_id = tup[5]
    matchup = tup[7]
    
    #home team main stats
    home_pts = tup[27]
    home_plusminus = tup[28]
    
    #secondary stats
    home_wl = tup[8]
    home_mins = tup[9]
    home_fgm = tup[10]
    home_fga= tup[11]
    home_fg_pct = tup[12]
    home_fg3m = tup[13]
    home_fg3a = tup[14]
    home_fg3_pct = tup[15]
    home_ftm = tup[16]
    home_fta = tup[17]
    home_ft_pct = tup[18]
    home_oreb = tup[19]
    home_dreb = tup[20]
    home_reb = tup[21]
    home_ast = tup[22]
    home_stl = tup[23]
    home_blk = tup[24]
    home_tov = tup[25]
    home_pf = tup[26]
    
    #away team
    away_pts = tup[53]
    away_plusminus = tup[54]
    
    #secondary stats
    away_wl = tup[34]
    away_mins = tup[35]
    away_fgm = tup[36]
    away_fga= tup[37]
    away_fg_pct = tup[38]
    away_fg3m = tup[39]
    away_fg3a = tup[40]
    away_fg3_pct = tup[41]
    away_ftm = tup[42]
    away_fta = tup[43]
    away_ft_pct = tup[44]
    away_oreb = tup[45]
    away_dreb = tup[46]
    away_reb = tup[47]
    away_ast = tup[48]
    away_stl = tup[49]
    away_blk = tup[50]
    away_tov = tup[51]
    away_pf = tup[52]
 
    #date
    d_full = dt.strptime(game_date, '%Y-%m-%d').strftime('%B %d, %Y')
    d_abb = dt.strptime(game_date, '%Y-%m-%d').strftime('%b %d, %Y')
    
    #construct query--2 queries to account for month abbrv.
    check_for = (hometeam, awayteam, d_full)
    check_for2 = (hometeam, awayteam, d_abb)
    
    query = (f'{thread} {hometeam} {awayteam} {d_full}')
    query2 = (f'{thread} {hometeam} {awayteam} {d_abb}')
    
    #initiate query
    print(f'searching {count}: "{query}"...'+'\n')
    search = subreddit.search(query, limit=25)
    
    for submission in search:
        Found = 0
        
        timestamp = dt.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M')
        
        print(f"{timestamp}, {submission.title}")
        print(submission.url)
        
        #check for phrases in title string
        if all(phrase in submission.title for phrase in check_for):
            print(f'---found all of {check_for}---'+'\n')
            
            #intial variable definition
            top_comment = submission.comments[0]
            top_comment_utc = dt.utcfromtimestamp(top_comment.created_utc).strftime('%Y-%m-%d %H:%M')
            expletive_score = predict([submission.comments[0].body])
            
            print(f'Top comment: {top_comment.body}\n')
            print(f'Expletive detection: {expletive_score}\n')
                         
            #create db doc
            document = (
                {'game_id': game_id,
                'season': game_season,
                'game_date': game_date,
                'matchup': matchup,
                'hometeam': hometeam,
                'home_pts': home_pts,
                'home_wl': home_wl,
                'awayteam': awayteam,
                'away_pts': away_pts,
                'away_wl': away_wl,
                'home_plusminus': home_plusminus,
                'away_plusminus': away_plusminus,
                'home_stats':
                    {'home_mins': home_mins,
                    'home_fgm': home_fgm,
                    'home_fga': home_fga,
                    'home_fg_pct': home_fg_pct,
                    'home_fg3m': home_fg3m,
                    'home_fg3a': home_fg3a,
                    'home_fg3_pct': home_fg3_pct,
                    'home_ftm': home_ftm,
                    'home_fta': home_fta,
                    'home_ft_pct': home_ft_pct,
                    'home_oreb': home_oreb,
                    'home_dreb': home_dreb,
                    'home_reb': home_reb,
                    'home_ast': home_ast,
                    'home_stl': home_stl,
                    'home_blk': home_blk,
                    'home_tov': home_tov,
                    'home_pf': home_pf,
                    }, 
                 'away_stats':
                    {'away_mins': away_mins,
                    'away_fgm': away_fgm,
                    'away_fga': away_fga,
                    'away_fg_pct': away_fg_pct,
                    'away_fg3m': away_fg3m,
                    'away_fg3a': away_fg3a,
                    'away_fg3_pct': away_fg3_pct,
                    'away_ftm': away_ftm,
                    'away_fta': away_fta,
                    'away_ft_pct': away_ft_pct,
                    'away_oreb': away_oreb,
                    'away_dreb': away_dreb,
                    'away_reb': away_reb,
                    'away_ast': away_ast,
                    'away_stl': away_stl,
                    'away_blk': away_blk,
                    'away_tov': away_tov,
                    'away_pf': away_pf,
                    },
                'game_thread':
                    {'id_submission': str(submission.id),
                    'created_utc': timestamp,
                    'title': str(submission.title),
                    'url': str(submission.url),
                    'num_comments': int(submission.num_comments),
                    'top_comment': 
                        {'id_comment': str(top_comment.id),
                        'author': str(top_comment.author),
                        'body': str(top_comment.body),
                        'score': int(top_comment.score),
                        'created_utc': str(top_comment_utc),
                        'permalink': str(top_comment.permalink),
                        'expletive_score': int(expletive_score)
                        },
                    }
                }
            )

            #replace_one upsert=True method
            db.nba_reddit.replace_one(document, document, True)  
              
            #count and move on
            count_found += 1
            found = True
            break
        
        print('---not a match---\n')
        found = False
    
    #secondary query--abbrv
    if found == False:
        print(f'Not found... searching {count}: "{query2}"...'+'\n')
        search = subreddit.search(query2, limit=25)
        
        for submission in search:
            
            print(f"{dt.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M')}, {submission.title}")
            print(submission.url)  
            
            #conditional      
            if all(phrase in submission.title for phrase in check_for2):
                print(f'---found all of {check_for2}---'+'\n')
                
                #intial variable definition
                top_comment = submission.comments[0]
                top_comment_utc = dt.utcfromtimestamp(top_comment.created_utc).strftime('%Y-%m-%d %H:%M')
                expletive_score = predict([submission.comments[0].body])

                print(f'Top comment: {top_comment.body}')
                print(f'Expletive detection: {expletive_score}\n')

                #create db doc
                document = (
                    {'game_id': game_id,
                    'season': game_season,
                    'game_date': game_date,
                    'matchup': matchup,
                    'hometeam': hometeam,
                    'home_pts': home_pts,
                    'home_wl': home_wl,
                    'awayteam': awayteam,
                    'away_pts': away_pts,
                    'away_wl': away_wl,
                    'home_plusminus': home_plusminus,
                    'away_plusminus': away_plusminus,
                    'home_stats':
                        {'home_mins': home_mins,
                        'home_fgm': home_fgm,
                        'home_fga': home_fga,
                        'home_fg_pct': home_fg_pct,
                        'home_fg3m': home_fg3m,
                        'home_fg3a': home_fg3a,
                        'home_fg3_pct': home_fg3_pct,
                        'home_ftm': home_ftm,
                        'home_fta': home_fta,
                        'home_ft_pct': home_ft_pct,
                        'home_oreb': home_oreb,
                        'home_dreb': home_dreb,
                        'home_reb': home_reb,
                        'home_ast': home_ast,
                        'home_stl': home_stl,
                        'home_blk': home_blk,
                        'home_tov': home_tov,
                        'home_pf': home_pf,
                        }, 
                     'away_stats':
                        {'away_mins': away_mins,
                        'away_fgm': away_fgm,
                        'away_fga': away_fga,
                        'away_fg_pct': away_fg_pct,
                        'away_fg3m': away_fg3m,
                        'away_fg3a': away_fg3a,
                        'away_fg3_pct': away_fg3_pct,
                        'away_ftm': away_ftm,
                        'away_fta': away_fta,
                        'away_ft_pct': away_ft_pct,
                        'away_oreb': away_oreb,
                        'away_dreb': away_dreb,
                        'away_reb': away_reb,
                        'away_ast': away_ast,
                        'away_stl': away_stl,
                        'away_blk': away_blk,
                        'away_tov': away_tov,
                        'away_pf': away_pf,
                        },
                    'game_thread':
                        {'id_submission': str(submission.id),
                        'created_utc': timestamp,
                        'title': str(submission.title),
                        'url': str(submission.url),
                        'num_comments': int(submission.num_comments),
                        'top_comment': 
                            {'id_comment': str(top_comment.id),
                            'author': str(top_comment.author),
                            'body': str(top_comment.body),
                            'score': int(top_comment.score),
                            'created_utc': str(top_comment_utc),
                            'permalink': str(top_comment.permalink),
                            'expletive_score': int(expletive_score)
                            },
                        }
                    }
                )

                #replace_one upsert=True method
                db.nba_reddit.replace_one(document, document, True)
                  
                #count and move on  
                count_found += 1
                found = True
                break
            
            print('---not a match---\n')
            found = False
              
    print('--------next game--------\n')
              
print(f'All done! {count_found}/{count} games found and uploaded to db')